In [121]:
import pandas as pd
from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob
import nltk
import numpy as np
# THIS DOWNLOAD WILL TAKE A FEW MINUITES BUT DO IT ONCE AND IT SHOULD BE FAST AFTER THAT
nltk.download("all")
from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
from nltk.tag import pos_tag_sents

#https://www.kaggle.com/ssishu/factual-authenticity-analysis-of-tweets

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/salvycavicchio/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/salvycavicchio/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/salvycavicchio/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/salvycavicchio/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/salvycavicchio/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/salvycavicchio/nltk_data...
[nltk_data]    |   P

In [131]:
#create dataframe
tweets = pd.read_csv("../tweet_data.csv")
df = pd.DataFrame(data=tweets["Text"])
df.insert(1,column="retweets",value=tweets["retweets"])
# change to string
df["Text"] = df['Text'].astype(str)
df["retweets"] = df['retweets'].astype(int)


In [132]:
# clean the text data
#remove special characters
#df['Text'] = df['Text'].str.replace('[^ws]','')
#lowercase everything
df['Text'] = df['Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
# remove unimportant kinds of words that dont add value
unimportant = stopwords.words('english')
# not going to do stemming because this is twitter and people mispell things all the time
#acutlaly, here we can use textblob to correct the words spelling?

df.apply()

print(df['Text'])


TypeError: apply() missing 1 required positional argument: 'func'

In [125]:
polarity = np.zeros(len(df))
subjectivity = np.zeros(len(df))

for item in range(len(df)):
    polarity[item] = TextBlob(df['Text'][item]).sentiment[0]
    subjectivity[item] = TextBlob(df['Text'][item]).sentiment[1]
    
df.insert(2,column="polarity",value=polarity)
df.insert(3,column="subjectivity",value=subjectivity)
#print(df)

In [64]:
df.to_csv('sentiment_tweet_data_textblob.csv')

In [44]:
# now we pull in a beefy training set for the sentiment
sentimentTrainingData = pd.read_csv("../Kaggle/trainingSet.csv",encoding='latin-1', header=None)
sentimentTrainingData.columns = ["sentiment", "id", "date", "query", "user", "text"]
sentimentTrainingData = sentimentTrainingData.drop(["id", "date", "query", "user"], axis = 1)
sentimentTrainingData.head() 
sentimentTrainingData.sentiment.value_counts()

4    800000
0    800000
Name: sentiment, dtype: int64

In [45]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize # import word_tokenize

tok = WordPunctTokenizer()

In [46]:
# this section will be used in a tweet cleaner function

pat1 = r'@[A-Za-z0-9_]+'        # remove @ mentions fron tweets
pat2 = r'https?://[^ ]+'        # remove URL's from tweets
combined_pat = r'|'.join((pat1, pat2)) #addition of pat1 and pat2
www_pat = r'www.[^ ]+'         # remove URL's from tweets
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",   # converting words like isn't to is not
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner(text):  # define tweet_cleaner function to clean the tweets
    soup = BeautifulSoup(text, 'lxml')    # call beautiful object
    souped = soup.get_text()   # get only text from the tweets 
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")    # remove utf-8-sig codeing
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed) # calling combined_pat
    stripped = re.sub(www_pat, '', stripped) #remove URL's
    lower_case = stripped.lower()      # converting all into lower case
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case) # converting word's like isn't to is not
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)       # will replace # by space
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1] # Word Punct Tokenize and only consider words whose length is greater than 1
    return (" ".join(words)).strip() # join the words

In [38]:
nums = [0,400000,800000,1200000,1600000] # used for batch processing tweets
#nums = [0, 9999]
clean_tweet_texts = [] # initialize list
for i in range(nums[0],nums[4]): # batch process 1.6 million tweets                                                               
    clean_tweet_texts.append(tweet_cleaner(sentimentTrainingData['text'][i]))  # call tweet_cleaner function and pass parameter as all the tweets to clean the tweets and append cleaned tweets into clean_tweet_texts list

In [47]:
word_tokens = [] # initialize list for tokens
for word in clean_tweet_texts:  # for each word in clean_tweet_texts
    word_tokens.append(word_tokenize(word)) #tokenize word in clean_tweet_texts and append it to word_tokens list
    
df1 = [] # initialize list df1 to store words after lemmatization
from nltk.stem import WordNetLemmatizer # import WordNetLemmatizer from nltk.stem
lemmatizer = WordNetLemmatizer() # create an object of WordNetLemmatizer
for l in word_tokens: # for loop for every tokens in word_token
    b = [lemmatizer.lemmatize(q) for q in l] #for every tokens in word_token lemmatize word and giev it to b
    df1.append(b) #append b to list df1

clean_df1 =[] # initialize list clean_df1 to join word tokens after lemmatization
for c in df1:  # for loop for each list in df1
    a = " ".join(c) # join words in list with space in between and giev it to a
    clean_df1.append(a) # append a to clean_df1
    
cleanTrainingSet = pd.DataFrame(clean_df1,columns=['text']) # convert clean_tweet_texts into dataframe and name it as clean_df
cleanTrainingSet['target'] = sentimentTrainingData.sentiment # from earlier dataframe get the sentiments of each tweet and make a new column in clean_df as target and give it all the sentiment score
#clean_df

NameError: name 'clean_df' is not defined

In [50]:
cleanTrainingSet['clean_len'] = [len(t) for t in cleanTrainingSet.text] # Again make a new coloumn in the dataframe and name it as clean_len which will store thw number of words in the tweet
X = cleanTrainingSet.text # get all the text in x variable
y = cleanTrainingSet.target # get all the sentiments into y variable

In [56]:
import sklearn
from sklearn.naive_bayes import MultinomialNB # import Multinomial Naive Bayes model from sklearn.naive_bayes
from sklearn.feature_extraction.text import TfidfVectorizer #  import TF-idf vectorizer

In [57]:
vect = TfidfVectorizer(analyzer = "word", ngram_range=(1,3)) # Get Tf-idf object and save it as vect. We can select features from here we just have simply change 
                                                                                     #the ngram range to change the features also we can remove stop words over here with the help of stop parameter

In [58]:
from sklearn.model_selection import train_test_split #from sklearn.cross_validation import train_test_split to split the data into training and tesing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state= 1) # split the data into traing and testing set where ratio is 80:20


# X_train is the tweets of training data, X_test is the testing tweets which we have to predict, y_train is the sentiments of tweets in the traing data and y_test is the sentiments of the tweets  which we will use to measure the accuracy of the model


vect.fit(X_train) # fit or traing data tweets to vect
X_train_dtm = vect.transform(X_train) # transform our training data tweets
X_test_dtm = vect.transform(X_test)# transform our testing data tweets

from sklearn.naive_bayes import MultinomialNB # import Multinomial Naive Bayes model from sklearn.naive_bayes
nb = MultinomialNB(alpha = 10) # get object of Multinomial naive bayes model with alpha parameter = 10

nb.fit(X_train_dtm, y_train)# fit our both traing data tweets as well as its sentiments to the multinomial naive bayes model

from sklearn.model_selection import cross_val_score  # import cross_val_score from sklear.model_selection
accuracies = cross_val_score(estimator = nb, X = X_train_dtm, y = y_train, cv = 10) # do K- fold cross validation on our traing data and its sentimenst with 10 fold cross validation
accuracies.mean() # measure the mean accuray of 10 fold cross validation

y_pred_nb = nb.predict(X_test_dtm) # predict the sentiments of testing data tweets

from sklearn import metrics # import metrics from sklearn
metrics.accuracy_score(y_test, y_pred_nb) # measure the accuracy of our model on the testing data


0.796921875

In [69]:
import pickle
filename = 'finalized_sentiment_model.sav'
pickle.dump(nb, open(filename, 'wb'))

In [115]:
import pickle
filename = 'finalized_sentiment_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
y_pred_loaded = loaded_model.predict(X_test_dtm)

metrics.accuracy_score(y_test, y_pred_nb) 

0.796921875

In [114]:
tweet_t = vect.transform(df["Text"])

sklearn_sentiment_tweet_data = loaded_model.predict(tweet_t)

simple_scores = df[df.columns[2]].to_numpy()

for i in range(len(simple_scores)):
    if simple_scores[i] >= 0:
        simple_scores[i] = 4
    else:
        simple_scores[i]=0

print(simple_scores)

print(sklearn_sentiment_tweet_data)

metrics.accuracy_score(simple_scores, sklearn_sentiment_tweet_data) 

[4. 4. 4. ... 0. 0. 0.]
[4 4 0 ... 0 0 0]


0.5186666666666667